In [1]:
cd ..

/home/andv/important/chatbot_rag


/home/andv/important/chatbot_rag/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:

from sentence_transformers import CrossEncoder, SentenceTransformer, util
import torch
import numpy as np
import pandas as pd
import textwrap
import time
import sys
import locale
import os
from dotenv import load_dotenv

load_dotenv()
# Load models and data once
embedding_path = os.getenv("PROJECTCB1_EMBEDDING_MODEL")
embedding_model = SentenceTransformer(model_name_or_path=embedding_path, device="cuda")

# pr_model_path = os.getenv("POST_RETRIEVAL_MODEL")
# pr_model = CrossEncoder(model_name=pr_model_path, device="cuda")
# Load embeddings and data once
text_chunks_and_embedding_df = pd.read_csv(os.getenv("PROJECTCB1_EMBEDDING_DATA_PATH"))
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df[
    "embedding"
].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")
embeddings = torch.tensor(
    np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32
).to("cuda")


# Define helper function to print wrapped text
def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)


# Function to retrieve relevant resources
def retrieve_relevant_resources(query: str, n_resources_to_return: int = 5):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    # Use cosine similarity instead of dot score
    cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    scores, indices = torch.topk(input=cosine_scores, k=n_resources_to_return)
    context_items = [pages_and_chunks[i] for i in indices]

    results = [item["Final_Answer"] for item in context_items]
    # result = results[0]
    # score = scores.item()
    # print(ques_items)

    return results, scores

/home/andv/important/chatbot_rag/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
query = "nội dung dự án 1 chương trình mục tiêu quốc gia  "
n_resources_to_return = 5
a, b = retrieve_relevant_resources(query, n_resources_to_return)

In [4]:
a

['Dự án thành phần của Chương trình mục tiêu quốc gia bao gồm: dự án, tiểu dự án thuộc Chương trình mục tiêu quốc gia phát triển kinh tế - xã hội vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2021 - 2030, Chương trình mục tiêu quốc gia giảm nghèo bền vững giai đoạn 2021 - 2025; nội dung thành phần, nội dung thuộc Chương trình mục tiêu quốc gia xây dựng nông thôn mới giai đoạn 2021 - 2025.\n (Theo Điều 3, khoản 1 Nghị Quyết của Quốc Hội 111/2024/QH15)',
 '"1. Hồ sơ thẩm định chương trình, dự án bao gồm:\na) Tờ trình thẩm định chương trình, dự án;\nb) Báo cáo nghiên cứu khả thi chương trình, dự án;\nc) Các tài liệu khác có liên quan.\n2. Hồ sơ quyết định chương trình, dự án, nội dung, thời gian thẩm định, quyết định chương trình, dự án thực hiện theo quy định của Chính phủ." (Theo Điều 45 Luật, Bộ Luật 39/2019/QH14)',
 'Nghị định này quy định chi tiết một số nội dung về quản lý dự án đầu tư xây dựng.',
 'Dự án thành phần bao gồm: dự án, tiểu dự án thuộc Chương trình mục tiêu quốc 

In [5]:
b

tensor([0.2486, 0.2432, 0.2407, 0.2175, 0.2026], device='cuda:0')